# Intermovie
* La liste des acteurs par film
* La liste des films Américains (en gardant leur nom en français) et leur note moyenne
* Les notes moyennes des différents genres
* La note moyenne de chaque acteur par rapport aux films dans lesquels il apparaît
Imports and useful functions

## Imports and useful functions

In [7]:
import pandas as pd
import random

#TODO filter column csv
#TODO idempotence

def skip_idx(file):
    """Function to sample a csv file."""
    # Count the lines
    num_lines = sum(1 for l in open(file, encoding='utf-8'))
    print("File size", num_lines)

    # Sample size - in this case ~10%
    size = int(num_lines / 10)

    # The row indices to skip - make sure 0 is not included to keep the header!
    random.seed(30)
    skip_idx = random.sample(range(1, num_lines), num_lines - size)

    return skip_idx

## La liste des acteurs par film

## La liste des films Américains (en gardant leur nom en français) et leur note moyenne

In [8]:
df_title_akas = pd.read_csv('../data/title.akas.tsv', sep='\t', skiprows=skip_idx('../data/title.akas.tsv'), usecols=['titleId', 'title', 'region'])

File size 19527972


In [9]:
df_title_akas.head()

,titleId,title,region
0,tt0000001,Carmencita,\N
1,tt0000003,Szegény Pierrot,HU
2,tt0000003,Poor Pierrot,GB
3,tt0000004,Полная кружка пива,RU
4,tt0000005,The Blacksmith's Forge,GB


### Films Américains

In [10]:
df_us = df_title_akas[df_title_akas['region'] == 'US']
df_us.head()

,titleId,title,region
5,tt0000009,Miss Jerry,US
18,tt0000026,The Messers. Lumière at Cards,US
20,tt0000029,Baby's Dinner,US
28,tt0000045,The Washerwomen,US
34,tt0000076,Rip and the Dwarf,US


In [12]:
df_title_basics = pd.read_csv('../data/title.basics.tsv', sep='\t', skiprows=skip_idx('../data/title.basics.tsv'), usecols=['tconst', 'originalTitle'])

File size 6321303


In [13]:
df_title_basics.head()

,tconst,originalTitle
0,tt0000006,Chinese Opium Den
1,tt0000013,Neuville-sur-Saône: Débarquement du congrès de...
2,tt0000016,Barque sortant du port
3,tt0000022,Les forgerons
4,tt0000027,Place des Cordeliers à Lyon


In [18]:
df_title_ratings = pd.read_csv('../data/title.ratings.tsv', sep='\t', skiprows=skip_idx('../data/title.ratings.tsv'))
merge = df_us.merge(df_title_basics, left_on='titleId', right_on='tconst').merge(df_title_ratings)
merge.head()

,titleId,title,region,tconst,originalTitle,averageRating,numVotes
0,tt0000886,"Hamlet, Prince of Denmark",US,tt0000886,Hamlet,5.2,19
1,tt0005758,Mixed Up,US,tt0005758,Mixed Up,7.8,12
2,tt0013304,The Last Laugh,US,tt0013304,The Last Laugh,8.3,29
3,tt0014417,The Wheel,US,tt0014417,La roue,7.4,1679
4,tt0015191,On Time!,US,tt0015191,On Time,4.7,11
